In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
# import gym
import pickle
from collections import deque
import time
# import seaborn as sns

In [ ]:
import numpy as np

# Assuming your file is named 'your_data.npy'
file_path = './weaving_noisy.npy'

# Step 1: Read the .npy file
try:
    data = np.load(file_path)
except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
    # Create some dummy data for demonstration if the file doesn't exist
    data = np.random.rand(4, 300, 4)
    print("Using dummy data for demonstration.")

print(f"Original data shape: {data.shape}")

# Step 2: Extract the last trajectory
# In Python, list/array indexing is 0-based.
# To get the last element of an array, you can use index -1.
last_trajectory = data[0]


print(f"Shape of the last trajectory: {last_trajectory.shape}")



In [ ]:
class PeriodicDMP:
    def __init__(self, n_dof=2, n_basis=20, tau=1.0, alpha=25.0, beta=10.0):
        self.n_dof = n_dof
        self.n_basis = n_basis
        self.tau = tau
        self.alpha = alpha
        self.beta = beta
 
        self.time = np.linspace(0, tau, 300)
        self.psi = self._compute_basis_functions(self.time)
 
        self.weights = np.random.randn(self.n_basis, self.n_dof)
 
    def _compute_basis_functions(self, time):
 
        centers = np.linspace(0, 1, self.n_basis)
        psi = np.exp(-self.alpha * (time[:, None] - centers[None, :]) ** 2)
        return psi
 
    def fit(self, trajectory):
        goal_position = trajectory[-1]
 
        for i in range(self.n_dof):
            self.weights[:, i] = np.linalg.lstsq(self.psi, trajectory[:, i], rcond=None)[0]
 
    def generate(self, goal_position):
        y = np.zeros((len(self.time), self.n_dof))
        dy = np.zeros((len(self.time), self.n_dof))
 
        for i in range(len(self.time)):
            for j in range(self.n_dof):
                y[i, j] = np.dot(self.psi[i, :], self.weights[:, j])
 
            dy[i] = np.gradient(y[i], self.time[i])
 
        return y, dy
    

In [ ]:
child4 = last_trajectory  # Extract x and y coordinates
example_trajectory = child4
dmp = PeriodicDMP(n_dof=2)
 
dmp.fit(example_trajectory)
 
goal_position = example_trajectory[-1]
 
target_trajectory, _ = dmp.generate(goal_position)
 
plt.plot(target_trajectory[:, 0], target_trajectory[:, 1], label='Generated by DMP', color='r')
plt.plot(example_trajectory[:, 0], example_trajectory[:, 1], label='Original Trajectory', color='b', linestyle='dashed')
plt.title("Original vs. Generated Trajectory")
plt.legend()
plt.axis('equal')
plt.show()

In [ ]:
import numpy as np
from scipy.stats import gaussian_kde
 
# Assume you have your learned DMP and generated lots of trajectories to estimate visitation
n_rollouts = 100
all_points = []
 
for _ in range(n_rollouts):
    traj, _ = dmp.generate(goal_position)
    all_points.append(traj)
 
all_points = np.vstack(all_points)
 
# Fit KDE for visitation distribution
kde = gaussian_kde(all_points.T)
 
def compute_reward(trajectory, kde):
    probs = kde(trajectory.T)
    return np.sum(probs)
 


In [ ]:
# 3) Load rollouts and compute rewards
# -------------------------
base_path = "./weaving/"

rollout_files = {
    "BC": "bc_rollouts_cleaned.npy",
    "BC (noisy)": "bc_rollouts_noisy.npy",
    "Traj-BC": "traj-bc_rollouts_cleaned.npy",
    "Traj-BC (noisy)": "traj-bc_rollouts_noisy.npy",
    "ILEED": "ileed_rollouts_cleaned.npy",
    "ILEED (noisy)": "ileed_rollouts_noisy.npy",
    "NeuralODE": "neural-ode_rollouts_cleaned.npy",
    "NeuralODE (noisy)": "neural-ode_rollouts_noisy.npy",
    
}

results = {}

# Compute rewards
for model_name, file_name in rollout_files.items():
    file_path = os.path.join(base_path, file_name)
    if not os.path.exists(file_path):
        print(f"File {file_name} not found, skipping {model_name}")
        continue

    rollouts = np.load(file_path, allow_pickle=True)
    print(f"Loaded {len(rollouts)} rollouts for {model_name}")

    policy_rollouts = [traj["states"] for traj in rollouts]
    rewards = np.array([compute_reward(traj, kde) for traj in policy_rollouts])

    results[model_name] = {
        "rewards": rewards,
        "mean": np.mean(rewards),
        "std": np.std(rewards)
    }

    print(f"  Mean reward: {results[model_name]['mean']:.6f}")
    print(f"  Std reward : {results[model_name]['std']:.6f}")

# -------------------------
# 4) Normalize rewards to [0, 1] across all models
# -------------------------
all_rewards = np.hstack([results[m]["rewards"] for m in results.keys()])
reward_min = all_rewards.min()
reward_max = all_rewards.max()

for model_name in results.keys():
    rewards = results[model_name]["rewards"]
    rewards_norm = (rewards - reward_min) / (reward_max - reward_min)
    
    results[model_name]["rewards_norm"] = rewards_norm
    results[model_name]["mean_norm"] = np.mean(rewards_norm)
    results[model_name]["std_norm"] = np.std(rewards_norm)

    print(f"{model_name}: Mean normalized = {results[model_name]['mean_norm']:.3f}, "
          f"Std normalized = {results[model_name]['std_norm']:.3f}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# -------------------------
# Your data
# -------------------------
results = {
    "BC":              {"mean": 0.320, "std": 0.210},
    "BC (noisy)":      {"mean": 0.298, "std": 0.225},
    "Traj-BC":         {"mean": 0.356, "std": 0.233},
    "Traj-BC (noisy)": {"mean": 0.302, "std": 0.231},
    "ILEED":           {"mean": 0.422, "std": 0.120},
    "ILEED (noisy)":   {"mean": 0.338, "std": 0.116},
    "NODE":       {"mean": 0.695, "std": 0.264},
    "NODE (noisy)": {"mean": 0.518, "std": 0.107},
}

# -------------------------
# Plotting mean ± std
# -------------------------
plt.figure(figsize=(12, 6))

# Extract model names (excluding noisy versions for x-axis)
clean_models = [m for m in results.keys() if "(noisy)" not in m]
x = np.arange(len(clean_models))
bar_width = 0.35

# Extract clean and noisy data
clean_means = [results[m]["mean"] for m in clean_models]
clean_std   = [results[m]["std"]  for m in clean_models]

noisy_means = [results[m + " (noisy)"]["mean"] for m in clean_models]
noisy_std   = [results[m + " (noisy)"]["std"]  for m in clean_models]

# Make error bars bold using error_kw
error_kw = dict(elinewidth=2.5, capthick=2.5)  # controls thickness

plt.bar(
    x - bar_width/2, clean_means, width=bar_width, yerr=clean_std,
    capsize=5, label="Clean", color="skyblue", edgecolor="k", error_kw=error_kw
)
plt.bar(
    x + bar_width/2, noisy_means, width=bar_width, yerr=noisy_std,
    capsize=5, label="Noisy", color="salmon", edgecolor="k", error_kw=error_kw
)


# Formatting
plt.ylabel("Normalized Reward", fontsize=20,fontweight="bold")
# plt.title("Comparison of Clean vs Noisy Rollouts (Mean ± Std)", fontsize=16, fontweight="bold")
plt.xticks(x, clean_models, rotation=20, fontsize=16,fontweight="bold")
plt.yticks(fontsize=16,fontweight="bold")
# plt.legend(fontsize=16,fontweight="bold")
# Bold legend
legend = plt.legend(frameon=False, fontsize=16)
for text in legend.get_texts():
    text.set_fontweight("bold")

# Thicker spines
ax = plt.gca()
for spine in ax.spines.values():
    spine.set_linewidth(1.5)

plt.tight_layout()
plt.savefig("wmatrix.png", dpi=600, bbox_inches="tight")
plt.show()
# plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

# Create a grid
xmin, ymin = -2, -2
xmax, ymax = 2, 2
X, Y = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]
positions = np.vstack([X.ravel(), Y.ravel()])
Z = np.reshape(kde(positions), X.shape)

# Generate ticks automatically
xticks = np.linspace(xmin, xmax, 7)
yticks = np.linspace(ymin, ymax, 7)

plt.figure(figsize=(5, 5))
im = plt.imshow(np.rot90(Z), cmap=plt.cm.viridis,
                extent=[xmin, xmax, ymin, ymax])

plt.scatter(all_points[:, 0], all_points[:, 1], s=5, alpha=0.3, c="white")

plt.xticks(xticks, fontsize=16, fontweight='bold')
plt.yticks(yticks, fontsize=16, fontweight='bold')
ax = plt.gca()
ax.xaxis.set_major_formatter(FormatStrFormatter('%.1f'))
ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
plt.axis([xmin, xmax, ymin, ymax])

plt.title("Value Map", fontsize=16, fontweight='bold')

# cbar = plt.colorbar(im)
# cbar.set_label("State Visitation Rate", fontsize=14, fontweight='bold')
# Get tick labels and make them bold
# for label in cbar.ax.get_yticklabels():
#     label.set_fontsize(16)
#     label.set_fontweight('bold')

plt.savefig("wvalue_map.png", dpi=600, bbox_inches="tight")
plt.show()
